In [1]:
# Library
import urllib.request
import json
import time as tm
import io
from time import sleep
import datetime as dt

from coinmarketcap import Market

In [2]:
class MarketListener():
    """
    The MarketListener class is used to write the market from the coinmarketcap API
    """

    def __init__(self,api,time_limit,batch_market_filename,streaming_market_filename,batch_summary_market_filename,streaming_summary_market_filename):
        self.batch_market_file = io.open(batch_market_filename, 'a')
        self.streaming_market_file = io.open(streaming_market_filename, 'w')
        
        self.batch_summary_market_file = io.open(batch_summary_market_filename, 'a')
        self.streaming_summary_market_file = io.open(streaming_summary_market_filename, 'w')
        
        self.start_time = tm.time()
        self.limit = time_limit
        self.api = api
        
    def get_ticker(self): 
        """
        Get the 100 first tickers  
        """
        return self.api.ticker(limit=100)
    
    def get_summary(self):
        """
        Get the summary about the market
        """
        return self.api.stats()
        
    def on_data(self):
        """
        Write the market in the batch & streaming files
        """
        if (tm.time() - self.start_time) < self.limit:       
            ticker_market = MarketListener.get_ticker(self) 
            summary_market = MarketListener.get_summary(self) 
            
            self.batch_market_file.write(json.dumps(ticker_market))
            self.batch_market_file.write("\n")
            
            self.batch_summary_market_file.write(json.dumps(summary_market))
            self.batch_summary_market_file.write("\n")
            
            self.streaming_market_file.write(json.dumps(ticker_market))
            self.streaming_market_file.write("\n")
            
            self.streaming_summary_market_file.write(json.dumps(summary_market))
            self.streaming_summary_market_file.write("\n")
            
            tm.sleep(60)
            return True
        else:
            self.batch_market_file.close()
            self.streaming_market_file.close()
            
            self.batch_summary_market_file.close()
            self.streaming_summary_market_file.close()
            return False
        
    def streaming(self):
        """
        Define the streaming 
        """
        while MarketListener.on_data(self) == True:
            continue
            

In [ ]:
if __name__ == '__main__':
    
    ###
    api_market = Market()
    market_listener = MarketListener(api=api_market,time_limit=60,batch_market_filename='raw_market_batch3.json',streaming_market_filename='raw_market_streaming3.json',batch_summary_market_filename='raw_summary_market_batch3.json',streaming_summary_market_filename='raw_summary_market_streaming3.json')
    market = market_listener.streaming()